# Hexagonal Tiling for Cartograms

This notebook demonstrates how to create **hexagonal tile maps** from cartograms using pycartogram. The technique overlays a hexagonal grid on a density-equalized cartogram and assigns each hexagon to its corresponding region.

## Why Hexagonal Tiling?

Hexagonal tilings offer several advantages for visualizing geographic data:
- **Equal area representation**: Each hexagon represents the same visual weight
- **Consistent neighbors**: Each hexagon has exactly 6 neighbors (unlike squares with 4 or 8)
- **Reduces visual bias**: Eliminates the visual dominance of large, sparsely populated regions
- **Clean aesthetics**: Creates a uniform, modern look

In this example, we create a population density cartogram of German electoral districts and then convert it to a hexagonal tile representation.

## Imports

In [ ]:
from itertools import groupby
from copy import deepcopy

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
from shapely.strtree import STRtree

from pycartogram.geopandas_cartogram import GeoDataFrameWardCartogram
from pycartogram.tools import fix_geodataframe_geometries, fix_invalid_geometry
from pycartogram.hexgrid import get_hexgrid

# Figure settings
plt.rcParams['figure.dpi'] = 75
_FIGSIZE = (7, 10)

## 1. Loading the Data

We need two datasets:
- **Population data**: Population counts per electoral district
- **Geo data**: Electoral district boundaries (Wahlkreise)

In [ ]:
# Load population data
popdata = pd.read_csv('data/btw21_population.csv')
pop = {items[2]: items[-1] for items in popdata.to_records()}

# Load election statistics
statsdata = pd.read_csv('data/btw21_stats.csv')
statsdata = statsdata.sort_values(by='wkr_nr')

# Load electoral district geometries
gdf = gpd.read_file("data/Geometrie_Wahlkreise_20DBT_VG250.json", crs='EPSG:32633')
gdf = gdf.set_crs('EPSG:25832', allow_override=True)

print(f"Loaded {len(gdf)} electoral districts")
print(f"CRS: {gdf.crs}")

In [ ]:
# Calculate population density for each district
_pop_dens = []
_pop = []
for _id, pol in zip(gdf.WKR_NR.to_list(), gdf.geometry.to_list()):
    _pop_dens.append(pop[int(_id)] / pol.area)
    _pop.append(pop[int(_id)])

gdf['population_density'] = _pop_dens
gdf['population'] = _pop
gdf = gdf.sort_values(by='WKR_NR')

# Merge statistics data
for col in statsdata:
    gdf[col] = statsdata[col].to_list()

print(f"Population density range: {min(_pop_dens):.2e} to {max(_pop_dens):.2e}")

In [ ]:
# Visualize population density on the original map
ax = gdf.plot(column='population_density', cmap='magma_r', edgecolor='#555555', lw=0.3, figsize=_FIGSIZE)
ax.set_title('Population Density (Original Map)')
ax.axis('off')

## 2. Creating the Population Density Cartogram

A cartogram distorts geographic regions so their area reflects a variable of interest (here: population density). Urban areas expand while rural areas shrink.

In [ ]:
# Create the cartogram
carto = GeoDataFrameWardCartogram(
    gdf,
    'population_density',
    y_raster_size=2048,
    x_raster_size=768 * 2,
    map_orientation='portrait'
)

# Cast density to matrix (this prepares the diffusion algorithm)
carto.cast_density_to_matrix(verbose=True, set_boundary_to='400percent_min')

In [ ]:
# Visualize the density matrix
carto.plot(show_new_wards=False, show_density_matrix=True, figsize=_FIGSIZE)

In [ ]:
# Compute the cartogram transformation
carto.compute(verbose=True)
new_gdf = carto.get_cartogram_geo_df()

In [ ]:
# Visualize the cartogram result
ax = new_gdf.plot(column='population_density', edgecolor='#555555', cmap='magma_r', figsize=_FIGSIZE)
ax.set_title('Population Density Cartogram')
ax.axis('off')

## 3. Creating the Hexagonal Grid

We overlay a hexagonal grid on the cartogram. The grid size determines how many hexagons will represent the data.

In [ ]:
# Configure hexagon grid size (smaller = more hexagons)
HEXAGON_GRID_SIZE = 200

# Get the extent of the cartogram
ax = new_gdf.plot(column='population_density', edgecolor='w', cmap='autumn_r')
x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
plt.close()

# Generate hexagonal grid
polygons = get_hexgrid(x0, y0, x1, y1, (x1 - x0) / HEXAGON_GRID_SIZE)
hexdf = gpd.GeoDataFrame(polygons, columns=['geometry'])

print(f"Created {len(hexdf)} hexagons")

In [ ]:
# Visualize the hexagonal grid overlay on the cartogram
ax = new_gdf.plot(column='population_density', edgecolor='w', cmap='autumn_r', figsize=_FIGSIZE)
hexdf.plot(ax=ax, facecolor='None', edgecolor='#aaaaaa', lw=0.3)
ax.set_title('Hexagonal Grid Over Cartogram')
ax.axis('off')

## 4. Computing Hexagon-County Overlap

For each hexagon, we need to determine which electoral district it belongs to. We use a spatial index (STRtree) for efficient intersection queries and compute overlap areas.

In [ ]:
# Fix invalid geometries for intersection operations
target_polygons = [fix_invalid_geometry(g) for g in new_gdf['geometry']]
s = STRtree(target_polygons)
countyids = list(new_gdf['WKR_NR'])

# Compute overlap between each hexagon and all counties it intersects
overlap_data = []
for hex_id, source_polygon in hexdf.to_records():
    # STRtree.query returns indices in Shapely 2.0+
    result_indices = s.query(source_polygon)
    these_overlap_counties = []
    these_overlap_areas = []
    total_overlap_area = 0.0
    
    for i in result_indices:
        target_poly = target_polygons[i]
        countyid = countyids[i]
        overlap = source_polygon.intersection(target_poly)
        area = overlap.area
        if area > 0:
            total_overlap_area += area
            these_overlap_counties.append(countyid)
            these_overlap_areas.append(area)
    
    if total_overlap_area > 0:
        overlap_data.extend([
            (int(hex_id), int(countyid), float(area / total_overlap_area))
            for countyid, area in zip(these_overlap_counties, these_overlap_areas)
        ])

overlap_df = pd.DataFrame(overlap_data, columns=['hexid', 'WKR_NR', 'overlap'])
print(f"Computed {len(overlap_df)} hexagon-county overlap relationships")

## 5. Assigning Hexagons to Counties

Each hexagon is assigned to the county it overlaps with most. Some small counties might not get any hexagons initially - we handle this by "stealing" a hexagon from a neighboring county with many hexagons.

In [ ]:
# Assign each hexagon to the county with maximum overlap
idx = overlap_df.groupby(['hexid'])['overlap'].transform(max) == overlap_df['overlap']

hexagon_counties = [None for _ in range(len(hexdf))]
for hid, cid, _ in overlap_df[idx].to_records(index=False):
    hexagon_counties[hid] = cid

new_hexdf = hexdf.copy()
new_hexdf['WKR_NR'] = hexagon_counties

# Check for unassigned counties
assigned_counties = new_hexdf['WKR_NR'].unique()
assigned_counties = set(np.array(assigned_counties[~np.isnan(assigned_counties)], dtype=int).tolist())
all_counties = {int(c) for c in gdf['WKR_NR'].unique()}
unassigned_counties = all_counties - assigned_counties

print(f"Unassigned counties: {unassigned_counties}")

In [ ]:
# Handle unassigned counties by stealing hexagons from neighbors
if unassigned_counties:
    _hexagon = lambda x: x[0]
    _county = lambda x: x[1]
    _value = lambda x: x[2]
    
    # Assign a single county to each hexagon
    county_by_hex = {}
    data = sorted(overlap_data, key=_hexagon)
    for hex_id, grp in groupby(data, key=_hexagon):
        rows = [list(entry) for entry in grp]
        county_by_hex[hex_id] = _county(max(rows, key=_value))
    
    # Count hexagons per county
    data = sorted(county_by_hex.items(), key=_county)
    hexagon_count_by_county = {}
    for county, grp in groupby(data, key=_county):
        assigned_hexagons = [list(entry) for entry in grp]
        hexagon_count_by_county[county] = len(assigned_hexagons)
    
    # Group overlaps by county
    overlap_by_county = {}
    data = sorted(overlap_data, key=_county)
    for county, grp in groupby(data, key=_county):
        overlap_by_county[county] = [list(entry) for entry in grp]
    
    # Steal hexagons for unassigned counties
    for county in unassigned_counties:
        candidate_hexagons = [_hexagon(entry) for entry in overlap_by_county[county]]
        hex_ids = [
            (hex_id, county_by_hex[hex_id], hexagon_count_by_county[county_by_hex[hex_id]])
            for hex_id in candidate_hexagons
        ]
        this_hexagon = _hexagon(max(hex_ids, key=_value))
        hexagon_counties[this_hexagon] = county
        print(f"County {county} was assigned a hexagon from county {county_by_hex[this_hexagon]}")
    
    new_hexdf = hexdf.copy()
    new_hexdf['WKR_NR'] = hexagon_counties

## 6. Creating Hexagonal County Boundaries

We dissolve the hexagons by county ID to create smooth hexagonal boundaries for each electoral district.

In [ ]:
# Dissolve hexagons by county
hexa_counties = new_hexdf.dissolve(by='WKR_NR')

# Apply small buffer to close gaps between hexagons
hexa_counties_buf = hexa_counties.copy()
buffer_distance = np.sqrt(hexdf.iloc[0]['geometry'].area) / 10000000
buffered_polygons = [poly.buffer(buffer_distance) for poly in hexa_counties_buf['geometry']]
hexa_counties_buf['geometry'] = buffered_polygons

# Map back to original geodataframe structure
hexa_counties_buf.index = hexa_counties_buf.index.astype(int)
hexa_counties_buf_dict = dict(hexa_counties_buf.to_records())

new_geometry = [hexa_counties_buf_dict[int(cid)] for cid in new_gdf['WKR_NR']]
hexa_counties = new_gdf.copy()
hexa_counties['geometry'] = new_geometry

print(f"Created hexagonal boundaries for {len(hexa_counties)} counties")

In [ ]:
# Visualize the hexagonal tile map
ax = hexa_counties.plot(facecolor='w', edgecolor='#cccccc', lw=0.5, figsize=_FIGSIZE)

# Add state boundaries
gdf_valid = fix_geodataframe_geometries(hexa_counties)
gdf_valid.dissolve(by='LAND_NR').buffer(0.0001).plot(facecolor='None', edgecolor='#888888', ax=ax, lw=0.9)

ax.set_title('Hexagonal Tile Map')
ax.axis('off')

## 7. Comparing Original and Hexagonal Maps

Let's compare the original geographic map with the hexagonal tile representation.

In [ ]:
# Side-by-side comparison: Original vs Hexagonal
fig, axes = plt.subplots(1, 2, figsize=(14, 10))

# Original map
ax = axes[0]
gdf.plot(ax=ax, facecolor='w', edgecolor='#cccccc', lw=0.5)
gdf.dissolve(by='LAND_NR').buffer(0.0001).plot(facecolor='None', edgecolor='#888888', ax=ax, lw=0.9)
ax.set_title('Original Geographic Map')
ax.axis('off')

# Hexagonal tile map
ax = axes[1]
hexa_counties.plot(ax=ax, facecolor='w', edgecolor='#cccccc', lw=0.5)
gdf_valid.dissolve(by='LAND_NR').buffer(0.0001).plot(facecolor='None', edgecolor='#888888', ax=ax, lw=0.9)
ax.set_title('Hexagonal Tile Map')
ax.axis('off')

plt.tight_layout()

## 8. Saving Outputs

Export the hexagonal tile map as GeoJSON for use in other applications.

In [ ]:
# Save hexagonal tile map as GeoJSON
output_path = 'targetdata/WKR_cartogram_hexagons.json'
hexa_counties[['LAND_NR', 'WKR_NR', 'geometry']].to_crs("EPSG:4326").to_file(
    output_path, driver='GeoJSON'
)
print(f"Saved hexagonal tile map to: {output_path}")

## Summary

This notebook demonstrated the complete workflow for creating hexagonal tile maps:

1. **Load data**: Population and geographic data for electoral districts
2. **Create cartogram**: Distort geography to reflect population density
3. **Generate hex grid**: Overlay hexagons on the cartogram
4. **Compute overlaps**: Use spatial indexing (STRtree) for efficient intersection
5. **Assign hexagons**: Map each hexagon to its primary district
6. **Handle edge cases**: Ensure every district gets at least one hexagon
7. **Dissolve boundaries**: Create smooth hexagonal district shapes

### Key Parameters

- `HEXAGON_GRID_SIZE`: Controls hexagon density (smaller = more hexagons)
- `set_boundary_to`: Controls cartogram distortion intensity
- `y_raster_size` / `x_raster_size`: Resolution of the diffusion algorithm

### Applications

Hexagonal tile maps are especially useful for:
- Election result visualization
- Demographic data presentation
- Any data where equal visual representation matters more than geographic accuracy